In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
import os 

MONGO_USERNAME = os.getenv('MONGO_INITDB_ROOT_USERNAME')
MONGO_PASSWORD = os.getenv('MONGO_INITDB_ROOT_PASSWORD')
MONGO_HOST = os.getenv('MONGO_HOST')
MONGO_PORT = os.getenv('MONGO_INITDB_ROOT_PORT')

In [10]:
from pymongo import MongoClient

mongo_uri = f"mongodb://{MONGO_USERNAME}:{MONGO_PASSWORD}@{MONGO_HOST}:{MONGO_PORT}/"
client = MongoClient(mongo_uri)
db = client['llm_database']
collection = db['dataset_ai']

In [15]:
def add_task(task_data, model):
    task = {
        "model": model,
        "prompt": task_data,
        "status": "pending",
        "response": None
    }
    result = collection.insert_one(task)
    print(f"Added task with id: {result.inserted_id}")
    return result.inserted_id

In [16]:
import time


def wait_for_task_completion(task_id, timeout=300, check_interval=5):
    """
    Ожидает завершения задачи в течение заданного времени.
    
    :param task_id: ID задачи в базе данных
    :param timeout: Максимальное время ожидания (в секундах)
    :param check_interval: Интервал проверки статуса задачи (в секундах)
    :return: Результат выполнения задачи или сообщение об ошибке
    """
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        # Поиск задачи по ID в коллекции
        task = collection.find_one({"_id": task_id})
        
        if task:
            status = task.get('status')
            if status == 'completed':
                # Забираем response при завершении
                return task.get('response')
            elif status == 'failed':
                return {"Ошибка": "Задача завершилась с ошибкой"}
        else:
            return {"Ошибка": "Задача не найдена"}
        
        # Ждем перед следующей проверкой
        time.sleep(check_interval)
    
    return {"Ошибка": "Превышено время ожидания завершения задачи"}


In [17]:
def extract_information(text: str) -> dict:
    model = 'qwen2:72b-instruct-q4_0'
    try:
        # Добавляем задачу в базу
        task_id = add_task(text, model)
        
        # Ожидаем завершения задачи и забираем результат
        result = wait_for_task_completion(task_id)
        
        return result
    except Exception as e:
        print(f"Ошибка при обработке: {e}")
        return {"Ошибка": "Не удалось обработать запрос"}


In [19]:
extract_information("Давайте напишите моему боссу 123@email.ru")

Added task with id: 66e27d091298a709fe0491f3


{'response_class': '5',
 'date': None,
 'contacts': '123@email.ru',
 'need_more_info': False,
 'model_response': 'Спасибо за предоставленные контакты. Мы обязательно направим всю необходимую информацию на адрес 123@email.ru для вашего босса. Если возникнут дополнительные вопросы или потребуется помощь, не стесняйтесь обращаться! С уважением, Команда Napoleon IT. Отзывы'}